In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import numpy as np


from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md

from sklearn.metrics.pairwise import cosine_similarity as cs

from pytorch_metric_learning.distances import LpDistance, CosineSimilarity,SNRDistance

def EERf(results):
    results = np.array(results)
    genuine = results[results[:, 1] == 1][:, 0]
    impostor = results[results[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return stats_a.eer, stats_a.fmr100, stats_a.fmr1000, stats_a.fmr10000


def calculate_and_print_averages(y_train, results3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    fmr100_values = []
    fmr1000_values = []
    fmr10000_values = []
    ii = 0

    for i in results3.keys():
        re = EERf(results3[i])
        eer = re[0]
        fmr100 = re[1]
        fmr1000 = re[2]
        fmr10000 = re[3]
        
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, FMR1000 = {re[2]:.4f}, FMR10000 = {re[3]:.4f}, Count = {counts[ii]}")
        
        eer_values.append(eer)
        fmr100_values.append(fmr100)
        fmr1000_values.append(fmr1000)
        fmr10000_values.append(fmr10000)
        ii += 1

    # Calculate averages and standard deviations
    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    average_fmr100 = np.mean(fmr100_values) * 100
    std_fmr100 = np.std(fmr100_values) * 100
    average_fmr1000 = np.mean(fmr1000_values) * 100
    std_fmr1000 = np.std(fmr1000_values) * 100
    average_fmr10000 = np.mean(fmr10000_values) * 100
    std_fmr10000 = np.std(fmr10000_values) * 100

    # Print results
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")
    print(f"Final Average FMR100: {average_fmr100:.4f}")
    print(f"Final FMR100 Standard Deviation: {std_fmr100:.4f}")
    print(f"${average_fmr100:.2f} \\pm {std_fmr100:.2f}$")
    print(f"Final Average FMR1000: {average_fmr1000:.4f}")
    print(f"Final FMR1000 Standard Deviation: {std_fmr1000:.4f}")
    print(f"${average_fmr1000:.2f} \\pm {std_fmr1000:.2f}$")
    print(f"Final Average FMR10000: {average_fmr10000:.4f}")
    print(f"Final FMR10000 Standard Deviation: {std_fmr10000:.4f}")
    print(f"${average_fmr10000:.2f} \\pm {std_fmr10000:.2f}$")
    
    return average_eer, std_eer, average_fmr100, std_fmr100, average_fmr1000, std_fmr1000, average_fmr10000, std_fmr10000



import torch.nn.functional as F







import torch
import numpy as np

def compute_embedding_batch_two(x_test_batch, embedding_network, batch_size=100, device="cuda"):
    #print(x_test_batch.shape)
    return x_test_batch

In [2]:
import numpy as np

# Loading the files
x_test_e = np.load('x_test_e.npy')
y_test_e = np.load('y_test_e.npy')
s_test_e = np.load('s_test_e.npy')
h_test_e = np.load('h_test_e.npy')

x_test_v = np.load('x_test_v.npy')
y_test_v = np.load('y_test_v.npy')
s_test_v = np.load('s_test_v.npy')
h_test_v = np.load('h_test_v.npy')

# Combining the arrays
X_test = np.concatenate((x_test_e, x_test_v), axis=0)
Y_test = np.concatenate((y_test_e, y_test_v), axis=0)
S_test = np.concatenate((s_test_e, s_test_v), axis=0)
H_test = np.concatenate((h_test_e, h_test_v), axis=0)

In [16]:



import numpy as np

#uy= np.unique(Y_test)[:5]
#ynl = np.isin(Y_test,uy)


#X_test = X_test[ynl]
#Y_test = Y_test[ynl]
#S_test = S_test[ynl]

# Generate a sequence of indices from 0 to the length of one of the arrays
indices = np.arange(X_test.shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]

# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
# Define the number of sessions for enrollment and verification
num_enrollment_sessions = 2
num_verification_sessions = 1  # This can be adjusted as needed

for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    unique_sessions = np.unique(subject_sessions)
    
    # Ensure there are enough sessions available for both enrollment and verification
    if len(unique_sessions) < (num_enrollment_sessions + num_verification_sessions):
        continue

    #print(subject, len(unique_sessions))
    
    # Select sessions for enrollment and verification
    enrollment_sessions = unique_sessions[:num_enrollment_sessions]
    verification_sessions = unique_sessions[num_enrollment_sessions:num_enrollment_sessions + num_verification_sessions]
    
    # Extend the enrollment lists with data from selected sessions
    x_test_e_list.extend(x_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    y_test_e_list.extend(y_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    s_test_e_list.extend(s_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])

    # Extend the verification lists with data from selected sessions
    x_test_v_list.extend(x_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    y_test_v_list.extend(y_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    s_test_v_list.extend(s_test[subject_indices][np.isin(subject_sessions, verification_sessions)])


indices = np.arange(np.array(x_test_e_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays


# Convert lists back to numpy arrays
x_test_e = np.array(x_test_e_list)
y_test_e = np.array(y_test_e_list)
s_test_e = np.array(s_test_e_list)

x_test_e = x_test_e[indices]
y_test_e = y_test_e[indices]
s_test_e = s_test_e[indices]


indices = np.arange(np.array(x_test_v_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)


x_test_v = np.array(x_test_v_list)
y_test_v = np.array(y_test_v_list)
s_test_v = np.array(s_test_v_list)

# Use the shuffled indices to reorder the arrays
x_test_v = x_test_v[indices]
y_test_v = y_test_v[indices]
s_test_v = s_test_v[indices]

print("subjects:", len(np.unique(y_test_e)))
print(f"x_test_e: {x_test_e.shape}, y_test_e: {y_test_e.shape}, s_test_e: {s_test_e.shape}")
print(f"x_test_v: {x_test_v.shape}, y_test_v: {y_test_v.shape}, s_test_v: {s_test_v.shape}")

subjects: 99
x_test_e: (19800, 128), y_test_e: (19800,), s_test_e: (19800,)
x_test_v: (9900, 128), y_test_v: (9900,), s_test_v: (9900,)


In [7]:



import numpy as np

#uy= np.unique(Y_test)[:5]
#ynl = np.isin(Y_test,uy)


#X_test = X_test[ynl]
#Y_test = Y_test[ynl]
#S_test = S_test[ynl]

# Generate a sequence of indices from 0 to the length of one of the arrays
indices = np.arange(X_test.shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]

# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
# Define the number of sessions for enrollment and verification
num_enrollment_sessions = 2
num_verification_sessions = 1  # This can be adjusted as needed

abs_two = []

for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    unique_sessions = np.unique(subject_sessions)
    
    # Ensure there are enough sessions available for both enrollment and verification
    if len(unique_sessions) < (num_enrollment_sessions + num_verification_sessions):
        continue

    #print(subject, len(unique_sessions))
    
    # Select sessions for enrollment and verification
    enrollment_sessions = unique_sessions[:num_enrollment_sessions]
    verification_sessions = unique_sessions[num_enrollment_sessions:num_enrollment_sessions + num_verification_sessions]
    
    abs_two.append([enrollment_sessions[0],enrollment_sessions[1]])
    
    # Extend the enrollment lists with data from selected sessions
    x_test_e_list.extend(x_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    y_test_e_list.extend(y_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])
    s_test_e_list.extend(s_test[subject_indices][np.isin(subject_sessions, enrollment_sessions)])

    # Extend the verification lists with data from selected sessions
    x_test_v_list.extend(x_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    y_test_v_list.extend(y_test[subject_indices][np.isin(subject_sessions, verification_sessions)])
    s_test_v_list.extend(s_test[subject_indices][np.isin(subject_sessions, verification_sessions)])


indices = np.arange(np.array(x_test_e_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)

# Use the shuffled indices to reorder the arrays


# Convert lists back to numpy arrays
x_test_e = np.array(x_test_e_list)
y_test_e = np.array(y_test_e_list)
s_test_e = np.array(s_test_e_list)

x_test_e = x_test_e[indices]
y_test_e = y_test_e[indices]
s_test_e = s_test_e[indices]


indices = np.arange(np.array(x_test_v_list).shape[0])

# Shuffle the indices
np.random.shuffle(indices)


x_test_v = np.array(x_test_v_list)
y_test_v = np.array(y_test_v_list)
s_test_v = np.array(s_test_v_list)

# Use the shuffled indices to reorder the arrays
x_test_v = x_test_v[indices]
y_test_v = y_test_v[indices]
s_test_v = s_test_v[indices]

print("subjects:", len(np.unique(y_test_e)))
print(f"x_test_e: {x_test_e.shape}, y_test_e: {y_test_e.shape}, s_test_e: {s_test_e.shape}")
print(f"x_test_v: {x_test_v.shape}, y_test_v: {y_test_v.shape}, s_test_v: {s_test_v.shape}")

subjects: 99
x_test_e: (19800, 128), y_test_e: (19800,), s_test_e: (19800,)
x_test_v: (9900, 128), y_test_v: (9900,), s_test_v: (9900,)


In [14]:
import pandas as pd

# Data: The provided list of lists
data = [
    [1, 2], [1, 12], [1, 3], [1, 3], [1, 4], [1, 196], [1, 69], [1, 8], [1, 4], [1, 228],
    [1, 4], [1, 73], [1, 7], [1, 6], [1, 5], [1, 93], [1, 233], [1, 3], [1, 5], [1, 6],
    [1, 7], [1, 6], [1, 2], [1, 3], [1, 4], [1, 3], [1, 7], [1, 5], [1, 5], [1, 8],
    [1, 5], [1, 6], [1, 8], [1, 3], [1, 5], [1, 2], [1, 7], [1, 6], [1, 6], [1, 2],
    [1, 7], [1, 7], [1, 10], [1, 5], [1, 4], [1, 6], [1, 2], [1, 7], [1, 14], [1, 2],
    [1, 5], [1, 4], [1, 5], [1, 8], [1, 2], [1, 2], [1, 2], [1, 4], [1, 6], [1, 302],
    [1, 12], [1, 2], [1, 2], [1, 1289], [1, 7], [1, 2], [1, 6], [1, 6], [1, 3], [1, 5],
    [1, 5], [1, 6], [1, 5], [1, 2], [1, 2], [1, 3], [1, 6], [1, 4], [1, 6], [1, 2],
    [1, 4], [1, 6], [1, 2], [1, 6], [1, 7], [1, 3], [1, 6], [1, 2], [1, 3], [1, 3],
    [1, 4], [1, 2], [1, 2], [1, 3], [1, 2], [1, 4], [1, 4], [1, 5], [1, 5]
]

# Convert to a DataFrame for easier analysis
df = pd.DataFrame(data, columns=["Col1", "Col2"])

# Compute statistics
total_pairs = len(df)
counts_of_second_element = df["Col2"].value_counts()
count_diff_1 = df[df["Col2"] - df["Col1"] == 1].shape[0]
count_diff_2 = df[df["Col2"] - df["Col1"] == 2].shape[0]
count_diff_less_3 = df[(df["Col2"] - df["Col1"]).abs() < 3].shape[0]
count_diff_less_7 = df[(df["Col2"] - df["Col1"]).abs() < 8].shape[0]
count_diff_less_10 = df[(df["Col2"] - df["Col1"]).abs() < 11].shape[0]

# Display results
print("Total pairs:", total_pairs)
print("Counts of each unique second element:\n", counts_of_second_element)
print("Count with a difference of 1:", count_diff_1)
print("Count with a difference of 2:", count_diff_2)
print("Count within 2 days:", count_diff_less_3)
print("Count within 7 days:", count_diff_less_7)
print("Count within 10 days:", count_diff_less_10)

Total pairs: 99
Counts of each unique second element:
 Col2
2       20
6       16
5       14
3       12
4       12
7        9
8        4
12       2
69       1
196      1
73       1
228      1
93       1
233      1
10       1
14       1
302      1
1289     1
Name: count, dtype: int64
Count with a difference of 1: 20
Count with a difference of 2: 12
Count within 2 days: 32
Count within 7 days: 87
Count within 10 days: 88


In [32]:
def calculate_similarity_scores_two2(enrollment_embeddings,y_enrollment, verification_embeddings, y_verification,s_ver, extra= 3):
    flag_s=0
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    print(unique_classes)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    # Compute similarity matrix
    print("I am 9")
    #similarity_matrix =batch_pairwise_distances(verification_embeddings, enrollment_embeddings, 100)

    print(verification_embeddings.shape, enrollment_embeddings.shape )
    
    similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    #similarity_matrix = abs(cs(verification_embeddings, enrollment_embeddings))
    print("I am 10")
    print(similarity_matrix.shape)
    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        subject_session = s_ver[i]
        #current_ss= np.where((s_ver == subject_session) & (y_verification == current_class))
        
        current_ss = np.where((s_ver == subject_session) & (y_verification == current_class))[0]  # assuming s_ver == subject_session is an array of booleans
        #print("aa", len(current_ss))
        
        selected_indices = np.random.choice(current_ss, extra, replace=True)
        selected_indices = np.append(selected_indices, i)

        #print(current_class, y_verification[selected_indices], s_ver[selected_indices], selected_indices)

        predicted_scoresg = similarity_matrix[selected_indices]
        


        
        predicted_scores = similarity_matrix[i]


        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]


        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #same_class_indices = np.setdiff1d(same_class_indices, [i])
            #max_score = np.mean(predicted_scores[same_class_indices])
            #max_score = max(predicted_scores[same_class_indices])
            maxscore=[]
            for ir in range(extra+1):
                max_score = sum(sorted(predicted_scoresg[ir, same_class_indices], reverse=True)[:10]) / 10
                maxscore.append(max_score)
            max_score = sum(maxscore)/(extra+1)
            
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls,i, cls ])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls,i,i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls,i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two2(enrollment_data, ye, verification_data,yv, e_network, s_ver, distance = "cd"):
    x_enrollment, y_enrollment = enrollment_data , ye
    x_verification, y_verification = verification_data, yv


    print(f"Total number of enrollment samples: {len(x_enrollment)}")
    print(f"Total number of verification samples: {len(x_verification)}", verification_data.shape)

    # Compute embeddings for enrollment data
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    print("I am 0 - enrollment", enrollment_embeddings.shape)
    # Compute embeddings for verification data
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    print("I am 1")

    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two2(
        enrollment_embeddings,y_enrollment, verification_embeddings, y_verification, s_ver
    )

    return similarity_results_by_class, similarity_results_by_class_dict

In [33]:

#, s_test_b
resutls2,resutls3=assessment_model_data_two2(x_test_e,y_test_e, x_test_v,y_test_v,None, s_test_v, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

Total number of enrollment samples: 19800
Total number of verification samples: 9900 (9900, 128)
I am 0 - enrollment (19800, 128)
I am 1
[ 70  82  85  86  96 103 106 109 111 115 118 119 125 129 130 131 136 145
 156 159 169 173 174 175 176 181 183 184 187 191 192 194 197 199 201 203
 206 207 214 216 217 219 221 225 230 231 233 236 245 249 250 251 257 260
 261 262 265 268 270 277 281 282 284 286 291 296 299 301 303 306 310 314
 322 325 332 336 342 348 354 357 364 365 373 374 384 385 387 402 404 407
 413 414 415 418 419 422 423 433 436]
I am 9
(9900, 128) (19800, 128)
I am 10
(9900, 19800)
70: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.1500, FMR10000 = 0.2900, Count = 200
82: EER = 0.0004, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0100, Count = 200
85: EER = 0.0111, FMR100 = 0.3100, FMR1000 = 0.9900, FMR10000 = 1.0000, Count = 200
86: EER = 0.0034, FMR100 = 0.0000, FMR1000 = 0.5000, FMR10000 = 0.9400, Count = 200
96: EER = 0.0004, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0900

(2.6682642754071324,
 3.8405364500015327,
 23.050505050505052,
 33.471644485943756,
 44.30303030303031,
 38.9953259425116,
 56.76767676767677,
 37.8793481440047)

In [1]:
raw_data = """
70: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.1500, FMR10000 = 0.2900, Count = 200
82: EER = 0.0004, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0100, Count = 200
85: EER = 0.0111, FMR100 = 0.3100, FMR1000 = 0.9900, FMR10000 = 1.0000, Count = 200
86: EER = 0.0034, FMR100 = 0.0000, FMR1000 = 0.5000, FMR10000 = 0.9400, Count = 200
96: EER = 0.0004, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0900, Count = 200
103: EER = 0.0009, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.1500, Count = 200
106: EER = 0.0346, FMR100 = 0.2200, FMR1000 = 0.4000, FMR10000 = 0.5500, Count = 200
109: EER = 0.0101, FMR100 = 0.0200, FMR1000 = 0.2200, FMR10000 = 0.4500, Count = 200
111: EER = 0.0114, FMR100 = 0.0400, FMR1000 = 0.5500, FMR10000 = 0.7000, Count = 200
115: EER = 0.0909, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
118: EER = 0.0325, FMR100 = 0.5400, FMR1000 = 0.9300, FMR10000 = 0.9800, Count = 200
119: EER = 0.0324, FMR100 = 0.7500, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
125: EER = 0.1301, FMR100 = 0.6700, FMR1000 = 0.8000, FMR10000 = 0.8400, Count = 200
129: EER = 0.1527, FMR100 = 0.7300, FMR1000 = 0.8800, FMR10000 = 0.9600, Count = 200
130: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
131: EER = 0.0939, FMR100 = 0.8400, FMR1000 = 0.9700, FMR10000 = 1.0000, Count = 200
136: EER = 0.0147, FMR100 = 0.1400, FMR1000 = 0.9600, FMR10000 = 1.0000, Count = 200
145: EER = 0.1000, FMR100 = 0.8500, FMR1000 = 0.9900, FMR10000 = 1.0000, Count = 200
156: EER = 0.0002, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0200, Count = 200
159: EER = 0.0148, FMR100 = 0.4200, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
169: EER = 0.0118, FMR100 = 0.0200, FMR1000 = 0.1700, FMR10000 = 0.2500, Count = 200
173: EER = 0.0002, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0100, Count = 200
174: EER = 0.1300, FMR100 = 0.7000, FMR1000 = 0.9700, FMR10000 = 1.0000, Count = 200
175: EER = 0.0123, FMR100 = 0.0900, FMR1000 = 0.6400, FMR10000 = 0.7400, Count = 200
176: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
181: EER = 0.0222, FMR100 = 0.0900, FMR1000 = 0.4100, FMR10000 = 0.7200, Count = 200
183: EER = 0.0230, FMR100 = 0.2900, FMR1000 = 0.9800, FMR10000 = 0.9900, Count = 200
184: EER = 0.0206, FMR100 = 0.1300, FMR1000 = 0.5400, FMR10000 = 0.8800, Count = 200
187: EER = 0.0718, FMR100 = 0.9000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
191: EER = 0.0021, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.0600, Count = 200
192: EER = 0.0044, FMR100 = 0.0000, FMR1000 = 0.0300, FMR10000 = 0.1200, Count = 200
194: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
197: EER = 0.1700, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
199: EER = 0.0028, FMR100 = 0.0000, FMR1000 = 0.0400, FMR10000 = 0.1600, Count = 200
201: EER = 0.0200, FMR100 = 0.1100, FMR1000 = 0.4600, FMR10000 = 0.5500, Count = 200
203: EER = 0.0200, FMR100 = 0.0600, FMR1000 = 0.2300, FMR10000 = 0.3800, Count = 200
206: EER = 0.0037, FMR100 = 0.0000, FMR1000 = 0.0800, FMR10000 = 0.2400, Count = 200
207: EER = 0.0203, FMR100 = 0.1300, FMR1000 = 0.4900, FMR10000 = 0.6400, Count = 200
214: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.0900, FMR10000 = 0.3000, Count = 200
216: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.2300, FMR10000 = 0.4900, Count = 200
217: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
219: EER = 0.0129, FMR100 = 0.0600, FMR1000 = 0.3900, FMR10000 = 0.6600, Count = 200
221: EER = 0.0113, FMR100 = 0.0200, FMR1000 = 0.1700, FMR10000 = 0.4400, Count = 200
225: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.1200, FMR10000 = 0.3200, Count = 200
230: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.0900, FMR10000 = 0.1600, Count = 200
231: EER = 0.0300, FMR100 = 0.1000, FMR1000 = 0.3500, FMR10000 = 0.4900, Count = 200
233: EER = 0.0700, FMR100 = 0.9800, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
236: EER = 0.0345, FMR100 = 0.4500, FMR1000 = 0.7900, FMR10000 = 0.9600, Count = 200
245: EER = 0.0201, FMR100 = 0.0800, FMR1000 = 0.1500, FMR10000 = 0.4800, Count = 200
249: EER = 0.0349, FMR100 = 0.9500, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
250: EER = 0.0024, FMR100 = 0.0000, FMR1000 = 0.0200, FMR10000 = 0.0900, Count = 200
251: EER = 0.0016, FMR100 = 0.0000, FMR1000 = 0.1000, FMR10000 = 0.4900, Count = 200
257: EER = 0.0004, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0100, Count = 200
260: EER = 0.0119, FMR100 = 0.1100, FMR1000 = 0.4100, FMR10000 = 0.7500, Count = 200
261: EER = 0.1600, FMR100 = 0.9200, FMR1000 = 0.9900, FMR10000 = 1.0000, Count = 200
262: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
265: EER = 0.0706, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
268: EER = 0.0132, FMR100 = 0.0200, FMR1000 = 0.1400, FMR10000 = 0.4700, Count = 200
270: EER = 0.0304, FMR100 = 0.2300, FMR1000 = 0.8500, FMR10000 = 0.9100, Count = 200
277: EER = 0.0003, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0100, Count = 200
281: EER = 0.0046, FMR100 = 0.0000, FMR1000 = 0.6600, FMR10000 = 0.9600, Count = 200
282: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
284: EER = 0.0234, FMR100 = 0.1500, FMR1000 = 0.5600, FMR10000 = 0.6500, Count = 200
286: EER = 0.0010, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.0600, Count = 200
291: EER = 0.0319, FMR100 = 0.1400, FMR1000 = 0.5400, FMR10000 = 0.7100, Count = 200
296: EER = 0.0600, FMR100 = 0.8700, FMR1000 = 0.9900, FMR10000 = 1.0000, Count = 200
299: EER = 0.0022, FMR100 = 0.0000, FMR1000 = 0.0500, FMR10000 = 0.3700, Count = 200
301: EER = 0.0005, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0200, Count = 200
303: EER = 0.0007, FMR100 = 0.0000, FMR1000 = 0.0400, FMR10000 = 0.1200, Count = 200
306: EER = 0.0109, FMR100 = 0.0400, FMR1000 = 0.8600, FMR10000 = 1.0000, Count = 200
310: EER = 0.0104, FMR100 = 0.0200, FMR1000 = 0.5600, FMR10000 = 0.6800, Count = 200
314: EER = 0.0236, FMR100 = 0.2400, FMR1000 = 0.7700, FMR10000 = 0.9600, Count = 200
322: EER = 0.0030, FMR100 = 0.0000, FMR1000 = 0.0300, FMR10000 = 0.2600, Count = 200
325: EER = 0.0010, FMR100 = 0.0000, FMR1000 = 0.0400, FMR10000 = 0.5100, Count = 200
332: EER = 0.0228, FMR100 = 0.2400, FMR1000 = 0.7000, FMR10000 = 0.9100, Count = 200
336: EER = 0.0428, FMR100 = 0.9100, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
342: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
348: EER = 0.0641, FMR100 = 0.9900, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
354: EER = 0.0129, FMR100 = 0.0700, FMR1000 = 0.4300, FMR10000 = 0.6800, Count = 200
357: EER = 0.1548, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
364: EER = 0.0231, FMR100 = 0.0700, FMR1000 = 0.3900, FMR10000 = 0.7300, Count = 200
365: EER = 0.0238, FMR100 = 0.1600, FMR1000 = 0.5600, FMR10000 = 0.7900, Count = 200
373: EER = 0.0319, FMR100 = 0.3700, FMR1000 = 0.8100, FMR10000 = 0.9400, Count = 200
374: EER = 0.0007, FMR100 = 0.0000, FMR1000 = 0.0300, FMR10000 = 0.2900, Count = 200
384: EER = 0.0201, FMR100 = 0.2100, FMR1000 = 0.7400, FMR10000 = 0.8900, Count = 200
385: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
387: EER = 0.0011, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.3300, Count = 200
402: EER = 0.0114, FMR100 = 0.0200, FMR1000 = 0.2700, FMR10000 = 0.3600, Count = 200
404: EER = 0.0016, FMR100 = 0.0000, FMR1000 = 0.0200, FMR10000 = 0.0900, Count = 200
407: EER = 0.0602, FMR100 = 0.6700, FMR1000 = 0.9600, FMR10000 = 0.9800, Count = 200
413: EER = 0.0128, FMR100 = 0.0800, FMR1000 = 0.4900, FMR10000 = 0.9300, Count = 200
414: EER = 0.0124, FMR100 = 0.0900, FMR1000 = 0.7700, FMR10000 = 0.9300, Count = 200
415: EER = 0.0006, FMR100 = 0.0000, FMR1000 = 0.0400, FMR10000 = 0.1800, Count = 200
418: EER = 0.0704, FMR100 = 0.6400, FMR1000 = 0.8800, FMR10000 = 0.9600, Count = 200
419: EER = 0.0327, FMR100 = 0.6900, FMR1000 = 0.9500, FMR10000 = 1.0000, Count = 200
422: EER = 0.0214, FMR100 = 0.0600, FMR1000 = 0.7200, FMR10000 = 0.8400, Count = 200
423: EER = 0.0116, FMR100 = 0.0500, FMR1000 = 0.5400, FMR10000 = 0.8400, Count = 200
433: EER = 0.0009, FMR100 = 0.0000, FMR1000 = 0.0200, FMR10000 = 0.1800, Count = 200
436: EER = 0.0105, FMR100 = 0.0200, FMR1000 = 0.1300, FMR10000 = 0.3000, Count = 200
"""

In [3]:
import pandas as pd
import re
# Regular expression to extract the data
pattern = r"(\d+): EER = ([\d.]+), FMR100 = ([\d.]+), FMR1000 = ([\d.]+), FMR10000 = ([\d.]+), Count = (\d+)"

# Find all matches
matches = re.findall(pattern, raw_data)

# Convert matches into the desired full_data structure
full_data = [(int(m[0]), float(m[1]), float(m[2]), float(m[3]), float(m[4]), int(m[5])) for m in matches]

# Create DataFrame
df_full = pd.DataFrame(full_data, columns=["ID", "EER", "FMR100", "FMR1000", "FMR10000", "Count"])

# Calculate thresholds
thresholds_full = {
    "FMR10000 <= 0.20": (df_full["FMR10000"] <= 0.20).sum(),
    "FMR10000 <= 0.10": (df_full["FMR10000"] <= 0.10).sum(),
    "FMR10000 <= 0.05": (df_full["FMR10000"] <= 0.05).sum(),
    "FMR1000 <= 0.20": (df_full["FMR1000"] <= 0.20).sum(),
    "FMR1000 <= 0.10": (df_full["FMR1000"] <= 0.10).sum(),
    "FMR1000 <= 0.05": (df_full["FMR1000"] <= 0.05).sum(),
}

# Print results
for condition, count in thresholds_full.items():
    print(f"{condition}: {count}")


FMR10000 <= 0.20: 26
FMR10000 <= 0.10: 19
FMR10000 <= 0.05: 14
FMR1000 <= 0.20: 41
FMR1000 <= 0.10: 34
FMR1000 <= 0.05: 30


In [6]:
raw_data = """
70: EER = 0.0018, FMR100 = 0.0000, FMR1000 = 0.0200, FMR10000 = 0.0200, Count = 200
82: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
85: EER = 0.0101, FMR100 = 0.8000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
86: EER = 0.0009, FMR100 = 0.0000, FMR1000 = 0.0600, FMR10000 = 0.2200, Count = 200
96: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
103: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
106: EER = 0.0110, FMR100 = 0.0400, FMR1000 = 0.2300, FMR10000 = 0.2800, Count = 200
109: EER = 0.0001, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
111: EER = 0.0019, FMR100 = 0.0000, FMR1000 = 0.1500, FMR10000 = 0.4100, Count = 200
115: EER = 0.0714, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
118: EER = 0.0211, FMR100 = 0.6400, FMR1000 = 0.9400, FMR10000 = 0.9900, Count = 200
119: EER = 0.0149, FMR100 = 0.8900, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
125: EER = 0.0829, FMR100 = 0.8300, FMR1000 = 0.9400, FMR10000 = 0.9600, Count = 200
129: EER = 0.0917, FMR100 = 0.8600, FMR1000 = 0.9700, FMR10000 = 0.9900, Count = 200
130: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
131: EER = 0.0621, FMR100 = 0.9700, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
136: EER = 0.0105, FMR100 = 0.0400, FMR1000 = 0.9200, FMR10000 = 1.0000, Count = 200
145: EER = 0.0600, FMR100 = 0.9300, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
156: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
159: EER = 0.0108, FMR100 = 0.5400, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
169: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
173: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
174: EER = 0.0600, FMR100 = 0.7100, FMR1000 = 0.9700, FMR10000 = 0.9900, Count = 200
175: EER = 0.0101, FMR100 = 0.0200, FMR1000 = 0.6200, FMR10000 = 0.9500, Count = 200
176: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
181: EER = 0.0027, FMR100 = 0.0000, FMR1000 = 0.0700, FMR10000 = 0.2300, Count = 200
183: EER = 0.0112, FMR100 = 0.2900, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
184: EER = 0.0030, FMR100 = 0.0000, FMR1000 = 0.1300, FMR10000 = 0.3400, Count = 200
187: EER = 0.0335, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
191: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
192: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
194: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
197: EER = 0.1227, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
199: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
201: EER = 0.0104, FMR100 = 0.0200, FMR1000 = 0.1200, FMR10000 = 0.2500, Count = 200
203: EER = 0.0007, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.0300, Count = 200
206: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
207: EER = 0.0023, FMR100 = 0.0000, FMR1000 = 0.0500, FMR10000 = 0.2900, Count = 200
214: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
216: EER = 0.0001, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
217: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
219: EER = 0.0018, FMR100 = 0.0000, FMR1000 = 0.0400, FMR10000 = 0.1100, Count = 200
221: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
225: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
230: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
231: EER = 0.0020, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.0300, Count = 200
233: EER = 0.0400, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
236: EER = 0.0146, FMR100 = 0.1100, FMR1000 = 0.4300, FMR10000 = 0.8500, Count = 200
245: EER = 0.0011, FMR100 = 0.0000, FMR1000 = 0.0100, FMR10000 = 0.0100, Count = 200
249: EER = 0.0303, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
250: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
251: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
257: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
260: EER = 0.0042, FMR100 = 0.0000, FMR1000 = 0.1900, FMR10000 = 0.4700, Count = 200
261: EER = 0.1028, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
262: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
265: EER = 0.0516, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
268: EER = 0.0001, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
270: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.4300, FMR10000 = 0.6400, Count = 200
277: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
281: EER = 0.0047, FMR100 = 0.0000, FMR1000 = 0.3600, FMR10000 = 0.6400, Count = 200
282: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
284: EER = 0.0042, FMR100 = 0.0000, FMR1000 = 0.0500, FMR10000 = 0.3700, Count = 200
286: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
291: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.0400, FMR10000 = 0.1900, Count = 200
296: EER = 0.0343, FMR100 = 0.9100, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
299: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
301: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
303: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
306: EER = 0.0100, FMR100 = 0.0000, FMR1000 = 0.7200, FMR10000 = 0.9500, Count = 200
310: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.2500, FMR10000 = 0.5000, Count = 200
314: EER = 0.0110, FMR100 = 0.0500, FMR1000 = 0.4500, FMR10000 = 0.7300, Count = 200
322: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
325: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
332: EER = 0.0130, FMR100 = 0.0800, FMR1000 = 0.3700, FMR10000 = 0.8700, Count = 200
336: EER = 0.0248, FMR100 = 0.9700, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
342: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
348: EER = 0.0335, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
354: EER = 0.0040, FMR100 = 0.0000, FMR1000 = 0.0200, FMR10000 = 0.0400, Count = 200
357: EER = 0.1400, FMR100 = 1.0000, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
364: EER = 0.0028, FMR100 = 0.0000, FMR1000 = 0.0500, FMR10000 = 0.1700, Count = 200
365: EER = 0.0044, FMR100 = 0.0000, FMR1000 = 0.2600, FMR10000 = 0.4900, Count = 200
373: EER = 0.0044, FMR100 = 0.0000, FMR1000 = 0.4100, FMR10000 = 0.6900, Count = 200
374: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
384: EER = 0.0037, FMR100 = 0.0000, FMR1000 = 0.2900, FMR10000 = 0.5500, Count = 200
385: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
387: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
402: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
404: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
407: EER = 0.0300, FMR100 = 0.8900, FMR1000 = 1.0000, FMR10000 = 1.0000, Count = 200
413: EER = 0.0022, FMR100 = 0.0000, FMR1000 = 0.1900, FMR10000 = 0.6800, Count = 200
414: EER = 0.0049, FMR100 = 0.0000, FMR1000 = 0.5100, FMR10000 = 0.7500, Count = 200
415: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
418: EER = 0.0401, FMR100 = 0.5100, FMR1000 = 0.9700, FMR10000 = 0.9900, Count = 200
419: EER = 0.0233, FMR100 = 0.7800, FMR1000 = 0.9800, FMR10000 = 1.0000, Count = 200
422: EER = 0.0038, FMR100 = 0.0000, FMR1000 = 0.2200, FMR10000 = 0.4200, Count = 200
423: EER = 0.0100, FMR100 = 0.0100, FMR1000 = 0.1000, FMR10000 = 0.4000, Count = 200
433: EER = 0.0000, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0000, Count = 200
436: EER = 0.0003, FMR100 = 0.0000, FMR1000 = 0.0000, FMR10000 = 0.0800, Count = 200
"""

In [7]:
import pandas as pd
import re
# Regular expression to extract the data
pattern = r"(\d+): EER = ([\d.]+), FMR100 = ([\d.]+), FMR1000 = ([\d.]+), FMR10000 = ([\d.]+), Count = (\d+)"

# Find all matches
matches = re.findall(pattern, raw_data)

# Convert matches into the desired full_data structure
full_data = [(int(m[0]), float(m[1]), float(m[2]), float(m[3]), float(m[4]), int(m[5])) for m in matches]

# Create DataFrame
df_full = pd.DataFrame(full_data, columns=["ID", "EER", "FMR100", "FMR1000", "FMR10000", "Count"])

# Calculate thresholds
thresholds_full = {
    "FMR10000 <= 0.20": (df_full["FMR10000"] <= 0.20).sum(),
    "FMR10000 <= 0.10": (df_full["FMR10000"] <= 0.10).sum(),
    "FMR10000 <= 0.05": (df_full["FMR10000"] <= 0.05).sum(),
    "FMR1000 <= 0.20": (df_full["FMR1000"] <= 0.20).sum(),
    "FMR1000 <= 0.10": (df_full["FMR1000"] <= 0.10).sum(),
    "FMR1000 <= 0.05": (df_full["FMR1000"] <= 0.05).sum(),
}


# Print results
for condition, count in thresholds_full.items():
    print(f"{condition}: {count}")


FMR10000 <= 0.20: 50
FMR10000 <= 0.10: 47
FMR10000 <= 0.05: 46
FMR1000 <= 0.20: 60
FMR1000 <= 0.10: 55
FMR1000 <= 0.05: 52
